In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 글씨체 조정
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# Kaggle Bike Sharing Demand 

*   EDA --> Preprocess --> 회귀  --> Randomforest --> 10Percent 도전
*   항목 추가
# 데이터 분석 방법론

#0. 목적 정의 
 - 분석 목적의 명확화
 - 분석 방법 정의
 
#1. 데이터 모양 확인
 - 단순 시각화 : 목적에 따른 데이터 표현
 
#2. 이상치 확인 및 NA확인

#3. 데이터 전처리
 - NA/이상치 처리 : 전체 삭제 or 평균 대체 or 다른 방법론을 통한 값 예측
 
#4. 분석 전 EDA
 - 범주형
 - 연속형
 - 융합형 (범주+연속)

#5. 분석
 - 회귀
 - randomforest?
 - 10percent

#6. 결론 도출

Reference : 
 - https://www.kaggle.com/kwonyoung234/for-beginner
 - https://www.youtube.com/watch?v=woSEc5d_skE&feature
 

# 0. 목적 정의

1. 분석 목적 : 2011년도와 2012년도의 자전거 대여 수치에 따라 공란으로 나온 count값에 대한 예측

2. 분석 방법 : 데이터에 대한 EDA 후, 전처리, 분석 전 확인 및 회귀, RNN 등의 기법을 활용하여 예측

In [ ]:
#라이브러리 호출

import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import calendar
from scipy import stats
from datetime import datetime

# 1. 데이터 모양 확인

In [ ]:
train = pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
train.shape

In [ ]:
test = pd.read_csv('../input/bike-sharing-demand/test.csv', parse_dates=['datetime'])
test.shape

 * 데이터의 모양을 보고 연/월/일/시간/분(무의미) / 쉬는 날인지 아닌지에 대한 여부 추가

In [ ]:
import missingno as msno
msno.matrix(train, figsize=(10,5))

 * 전체 컬럼에 na값 없음

In [ ]:
train["year"] = train.datetime.dt.year
train["month"] = train.datetime.dt.month
train["day"] = train.datetime.dt.day
train["hour"] = train.datetime.dt.hour
train["minute"] = train.datetime.dt.minute
train["dayofweek"] = train.datetime.dt.dayofweek

train.shape

In [ ]:
test["year"] = test.datetime.dt.year
test["month"] = test.datetime.dt.month
test["day"] = test.datetime.dt.day
test["hour"] = test.datetime.dt.hour
test["minute"] = test.datetime.dt.minute
test["dayofweek"] = test.datetime.dt.dayofweek

test.shape

In [ ]:
def concate_year_month(datetime):
    return "{0}-{1}".format(datetime.year, datetime.month)

train["year_month"] = train["datetime"].apply(concate_year_month)
test["year_month"] = test["datetime"].apply(concate_year_month)

In [ ]:
print(train.head())
train.info()

In [ ]:
print(train.registered.head())
print(train.casual.head())
print(train['count'].head())

## Data 설명 

* datetime - hourly date + timestamp  
 - 2011-01-01 01:00:00 (연-월-일 1시 0분 0초)
 - 년/월/일/시 별 column 생성
* season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
 - 1 : 봄, 2 : 여름, 3 : 가을, 4 : 겨울
* holiday - whether the day is considered a holiday
 - 0 : 평일, 1 : 공휴일 여부(주말 제외)
* workingday - whether the day is neither a weekend nor holiday
 - 0 : 쉬는 날, 1: 평일
* weather 
 - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 맑음, 약간 구름
 - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 흐림, 안개 
 - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 약한 눈/비, 천둥
 - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
    - 강한 눈, 우박, 천둥, 안개
* temp - temperature in Celsius
 - 온도
* atemp - "feels like" temperature in Celsius
 - 체감 온도
* humidity - relative humidity
 - 습도
* windspeed - wind speed
 - 바람 속도
* casual - number of non-registered user rentals initiated
 - 처음 사용하는 사람의 수
* registered - number of registered user rentals initiated
 - 원래 등록했던 사람의 사용수
* count - number of total rentals = Test에는 없는 값--> 예측해야 할 값
 - 전체 대여 수 (casual + registered)
* dayofweek - The day of week
 - 0 : Mon, 1 : Tue, 2 : Wed, 3: Thu, 4 : Fri, 5 : Sat, 6 : Sun

In [ ]:
# 목적 데이터(count) 탐색
sns.boxplot(data=train, y='count', orient='v')

 * 탐색해야 할 데이터의 모양을 살펴봣을때, 극단치의 존재가 데이터의 모양을 변경하고 있는 것으로 확인 해당 데이터 전처리를 할 것인지 결정 필요

In [ ]:
#outlier 탐색 
# outlier = n_i 값과 평균가의 차이가 3*(표준편차)초과인 값

train_without_liers = train[np.abs(train['count'] - train['count'].mean()) <= (3*train['count'].std())]
print(train.shape)
print(train_without_liers.shape)

* 약 150개의 데이터를 지울 수 있음...적용 여부는 두가지 모두 돌려 본 후 비교

## 시간 데이터 탐색

In [ ]:
#연도별, 월별, 일자별, 시간별 시각화
figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
figure.set_size_inches(18,12)

sns.barplot(data=train, x='year', y='count', ax=ax1)
sns.barplot(data=train, x='month', y='count', ax=ax2)
sns.barplot(data=train, x='day', y='count', ax=ax3)
sns.barplot(data=train, x='hour', y='count', ax=ax4)

ax1.set(ylabel='count', title='count of year')
ax2.set(ylabel='count', title='count of month')
ax3.set(ylabel='count', title='count of day')
ax4.set(ylabel='count', title='count of hour')

* 2011년에 비해 2012년에는 약 50%이상의 증가를 했으며, 이는 사용자 풀이 늘어난 것을 확인 할 수 있다.
* 월별로 확인 했을 때, 겨울, 봄에는 사용률이 낮으며, 여름, 가을의 경우 크게 늘어난 것을 확인할 수 있다. 
* 일자별로 봤을 경우, 각 일자가 크게 다르지 않으므로 (유의미한 차이 없음) 일자에 대한 의미는 없는 것으로 판단
  - day = 19일까지 밖에 없음
* 시간대 별로 봤을 경우,출퇴근 시간(8시, 5/6시 등)에 몰려서 사용하며, 취침시간에는 사용률이 가장 저조한 것을 확인할 수 있다.

### 2011년 월별 ->12년 월별 데이터까지 시간순 확인

In [ ]:
fig, ax = plt.subplots(nrows=1)
fig.set_size_inches(18,4)

sns.barplot(data=train, x='year_month', y='count', ax=ax)

* 연도별 사용률의 증가는 뚜렷하며, 계절에 따라 사용률의 차이가 명확함을 알 수 있음

## 카테고리 데이터 Categorical Data (Season, Holiday, Working Day, Weather)

In [ ]:
#계절, 휴일여부, 평일, 날씨
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(18, 12)

sns.barplot(data=train, x='season', y='count', ax=ax1)
sns.barplot(data=train, x='holiday', y='count', ax=ax2)
sns.barplot(data=train, x='workingday', y='count', ax=ax3)
sns.barplot(data=train, x='weather', y='count',ax=ax4)

* 계절 
  * 1 : 1,2,3월 = Spring? 
  * 계절에 대한 탐색 필요
  * 봄에 오히려 사용 수가 가장 적은것에 대한 문제, 수정 필요

* 평일엔 오히려 높고 주말엔 더 쉬어야 하지 않는가? (집돌이의 예상)

* 날씨가 가장 안좋을 때 더 많이 탄다? 편차 바가 없는 것으로 보아 표본수가 매우 적을 것으로 예상 확인필요


In [ ]:
bad_weather = train[train['weather']==4]
bad_weather.head()

 * 예상과 같이 데이터는 하나밖에 없음
  - 습도는 높으나, 풍속은 강하지 않음
  - 이전 시간 데이터 확인 필요

In [ ]:
train[(train['year']==2012) & (train['month'] == 1) & (train['day']== 9) ]

 * 당일의 날씨가 매우 안좋았다는 것을 확인할 수 있음
 * 추가적으로 풍속이 0인 시간 앞뒤로 풍속이 차이가 크기 않다는 것을 확인 할 수 있음.
  - 차후 풍속 0 인 데이터에 대한 후처리 필요
  - 1. 계획상 randomforest를 돌려 확인한다
  - 2. 수학적으로 명확한 방법을 위하여 풍속이 0인 시간 전후값의 평균값으로 수정한 데이터 세트도 산출하여 비교할 필요성 있음

### Holiday 확인
* 일하는 날인지가 더 중요하지 않을까 

In [ ]:
train.dayofweek.value_counts()

In [ ]:
sns.barplot(data=train, x='dayofweek', y='count')

* 5, 6 =  주말, 생각보다 사용률이 매우 높음. 
* 평일만큼 주말에도 많이 탄다는 결론
* 일요일에 가장 적음 (유의미해 보이지는 않음)

## 수치형 데이터들에 대한 시각화
- 온도 (temp)
- 체감온도 (atemp)
- 습도 (humidity)
- 풍속 (windspeed)

In [ ]:
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(12,10)

sns.lineplot(data=train, x='temp', y='count', ax=ax1)
sns.lineplot(data=train, x='atemp', y='count', ax=ax2)
sns.lineplot(data=train, x='humidity', y='count', ax=ax3)
sns.lineplot(data=train, x='windspeed', y='count', ax=ax4)

* temp, atemp는 유사한 모양으로, 온도와 체감온도가 크게 다르지 않음을 예상할 수 있음
 - 그에 따라 사용률 또한 유사한 모양을 그리는 것을 확인할 수 있음
* 습도의 경우, 20미만의 수치에 대해 변동성(옅은 색)이 크게 없음을 보아, 극소수의 숫자로 나타나는 것으로 보임
* 풍속의 경우, 풍속이 낮을 때는 일관적인 변동폭을 보이지만, 퐁속이 높아질수록 사용률 변동이 매우 커짐 
 - 풍속에 대한 추가 설명 필요

## 풍속 데이터 탐색

In [ ]:
fig, axes = plt.subplots(nrows=2)
fig.set_size_inches(19,10)

plt.sca(axes[0])
plt.xticks(rotation=30, ha='right')
axes[0].set(ylabel='count', title='train windspeed')
sns.countplot(data=train, x='windspeed', ax=axes[0])

plt.sca(axes[1])
plt.xticks(rotation=30, ha='right')
axes[1].set(ylabel='count', title='test windspeed')
sns.countplot(data=test, x='windspeed', ax=axes[1])

* 예상과 같이 풍속이 0인 데이터가 매우 많음 
 - 실제로 바람이 0인지 측정이 안된 것인지 확인필요
 - 풍속이 0 인 데이터의 전/후로 데이터 값이 있다면 누락으로 판단
 - 없다면 실제 풍속이 0일 수 있을 듯

# 3. 데이터 전처리
 * outlier 처리 : 비정상적인값(편차 > 3*표준편차) 확인 및 처리
 * NA/이상치 처리 : 전체 삭제 or 평균 대체 or 다른 방법론을 통한 값 예측

## 계절 재정의
* 봄 : 3,4,5 
* 여름 : 6, 7, 8
* 가을 : 9, 10, 11
* 겨울 : 12, 1, 2

ref. https://ko.weatherspark.com/y/23912/%EB%AF%B8%EA%B5%AD-%EB%89%B4%EC%9A%95-%EC%A3%BC-%EB%89%B4%EC%9A%95%EC%9D%98-%EB%85%84%EC%A4%91-%ED%8F%89%EA%B7%A0-%EB%82%A0%EC%94%A8

In [ ]:
#train 및 test 데이터 모두 변경
def to_right_season(month):
  if month in [3, 4, 5]:
    return 1
  elif month in [6,7,8] :
    return 2
  elif month in [9,10, 11]:
    return 3
  else : return 4

train['season'] = train.month.apply(to_right_season)
test['season'] = test.month.apply(to_right_season)

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_size_inches(18,5)
sns.barplot(data=train, x='season', y='count', ax=ax)


* 예상과 같이 봄, 겨울에 사용률이 저조함

## 풍속데이터 전처리
* 풍속이 0인 데이터만 추려 연속되는지 확인하고, 연속되는 데이터들이 아니라면, 다음을 수행
* 풍속이 0인 데이터가 연속되면 randomforest 값 활용

In [ ]:
# 풍속이 0인 값에 대해 확인
train_wind0 = train[train['windspeed']==0]
train_wind0

* windspeed가 0인값은 연속적인것과 연속적이지 않은 것들이 섞여 있음
* 세부적으로 다음 세트로 나눔
 - train_rand = 풍속0에 대해 randomforest를 돌려 예측
 - train_mean = 풍속이 0인 데이터의 앞뒤 값의 평균
 - train_rmean = 연속되는 풍속0은 randomforest 값, 연속되지 않으면 앞뒤 평균

In [ ]:
# randomforest
from sklearn.ensemble import RandomForestClassifier

def predict_wind(data):
    
    #windspeed가 0인 값과 아닌 값 구분
    data_wind0 = data.loc[data['windspeed']==0]
    data_windnot0 = data.loc[data['windspeed']!=0]
    
    #예측할 때 사용할 feature = 컬럼들 (연도와 날짜는 상관이 없을 것으로 예측(1년 사이의 일이기 때문))
    wCol = ['season', 'weather', 'humidity', 'month', 'temp', 'atemp']
    
    #문자열로 변환 (분류를 활용하기에)
    data_windnot0["windspeed"] = data_windnot0['windspeed'].astype('str')
    
    
    # 랜덤포레스트 분류기를 사용한다.
    rfModel_wind = RandomForestClassifier()

    # wCol에 있는 피처의 값을 바탕으로 풍속을 학습시킨다.
    rfModel_wind.fit(data_windnot0[wCol], data_windnot0["windspeed"])

    # 학습한 값을 바탕으로 풍속이 0으로 기록된 데이터의 풍속을 예측한다.
    wind0Values = rfModel_wind.predict(X = data_wind0[wCol])

    # 값을 다 예측 후 비교해 보기 위해
    # 예측한 값을 넣어 줄 데이터 프레임을 새로 만든다.
    predictWind0 = data_wind0
    predictWindNot0 = data_windnot0
    
    #예측한 값을 풍속 0인 데이터에 취합
    predictWind0["windspeed"] = wind0Values
    
    #0인 데이터와 아닌 데이터 취합하여 새로운 데이터 생성
    data_rand = predictWindNot0.append(predictWind0)
    
    #실수 형태로 
    data_rand['windspeed'] = data_rand['windspeed'].astype('float')
    
    data_rand.reset_index(inplace=True)
    data_rand.drop('index', inplace=True, axis=1)
    
    return data_rand
    

In [ ]:
train_rand = predict_wind(train)

fig, ax1=  plt.subplots()
fig.set_size_inches(19, 10)

plt.sca(ax1)
plt.xticks(rotation=30, ha = 'right')
ax1.set(ylabel = 'count' , title = 'train wind speed')
sns.countplot(data=train_rand, x='windspeed', ax=ax1)

In [ ]:
#train_mean
def wind_mean(train):
    wind_mean1 = train
    for i in (train[train['windspeed']==0]).index:
        if (i>0):
            if (((train.loc[i-1, 'windspeed'] + train.loc[i+1,'windspeed'])/2) > 1) :
                wind_mean1.loc[i,'windspeed'] = (train.loc[i-1, 'windspeed'] + train.loc[i+1,'windspeed'])/2
    return wind_mean1

In [ ]:
train_mean = wind_mean(train)

In [ ]:
fig, ax1=  plt.subplots()
fig.set_size_inches(19, 10)

plt.sca(ax1)
plt.xticks(rotation=30, ha = 'right')
ax1.set(ylabel = 'count' , title = 'train wind speed')
sns.countplot(data=train_mean, x='windspeed', ax=ax1)

In [ ]:
# train_rmean
def rmean(data):
    #풍속이 0인 데이터와 아닌 데이터 분리
    wind0 = (data[data['windspeed']==0]).index
    for i in wind0 :
        if ((i-1, i+1) in wind0) :
            data.loc[i,'windspeed'] = train_rand.loc[i, 'windspeed']
        else:
            data.loc[i,'windspeed'] = train_mean.loc[i, 'windspeed']
    return data

In [ ]:
train_rmean = rmean(train)

In [ ]:
fig, ax1=  plt.subplots()
fig.set_size_inches(19, 10)

plt.sca(ax1)
plt.xticks(rotation=30, ha = 'right')
ax1.set(ylabel = 'count' , title = 'train wind speed')
sns.countplot(data=train_rmean, x='windspeed', ax=ax1)

* 거의 모든 0의 값이 연속된 것을 확인할 수 있음

# 4. 분석 전 EDA

* Feature Selection을 위한 EDA
* 각 데이터 별 count와 관련성 확인을 위한 시각화

In [ ]:
# 수치형 (randomforest)
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(12,10)

sns.lineplot(data=train_rand, x='temp', y='count', ax=ax1)
sns.lineplot(data=train_rand, x='atemp', y='count', ax=ax2)
sns.lineplot(data=train_rand, x='humidity', y='count', ax=ax3)
sns.lineplot(data=train_rand, x='windspeed', y='count', ax=ax4)

In [ ]:
#mean
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(12,10)

sns.lineplot(data=train_mean, x='temp', y='count', ax=ax1)
sns.lineplot(data=train_mean, x='atemp', y='count', ax=ax2)
sns.lineplot(data=train_mean, x='humidity', y='count', ax=ax3)
sns.lineplot(data=train_mean, x='windspeed', y='count', ax=ax4)

In [ ]:
#rand+mean
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(12,10)

sns.lineplot(data=train_rmean, x='temp', y='count', ax=ax1)
sns.lineplot(data=train_rmean, x='atemp', y='count', ax=ax2)
sns.lineplot(data=train_rmean, x='humidity', y='count', ax=ax3)
sns.lineplot(data=train_rmean, x='windspeed', y='count', ax=ax4)

* 평균값으로 조정할 경우 count의 변화량이 급격하게 커짐
* 특히 풍속이 낮을 때 count 값의 변화가 심함 다른 요인이 개입된 것이거나, 예측을 잘못했을 경우

In [ ]:
# categorical data
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sns.boxplot(data=train,y="count",orient="v",ax=axes[0][0])
sns.boxplot(data=train,y="count",x="season",orient="v",ax=axes[0][1])
sns.boxplot(data=train,y="count",x="hour",orient="v",ax=axes[1][0])
sns.boxplot(data=train,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="대여량")
axes[0][1].set(xlabel='Season', ylabel='Count',title="계절별 대여량")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="시간별 대여량")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="근무일 여부에 따른 대여량")

* 대여량의 극단치 값이 존재하는 것으로 확인
* 계절별로 모양은 유사하지만 극단치가 많은 것으로 보임
* 극단치를 제외한 대부분의 모양을 확인할 수 있음
* working Day 의 평균 값은 차이가 커보이지 않으나, 극단치의 값이 일하는날 더 많다는 것을 확인할 수 잇음

In [ ]:
#융합형 - 시각화
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5)
fig.set_size_inches(18,25)

sns.pointplot(data=train, x= "hour" , y = "count" , ax= ax1)
sns.pointplot(data=train, x= "hour", y="count", hue="workingday", ax=ax2)
sns.pointplot(data=train, x="hour", y="count", hue="dayofweek", ax=ax3)
sns.pointplot(data=train, x="hour", y="count", hue= "weather", ax=ax4)
sns.pointplot(data=train, x="hour", y= "count", hue="season", ax=ax5)

* 이전 EDA와 같이, 출퇴근 시간에 사용빈도가 가장 높고, 날씨가 좋고, 평일일때 가장 많이 사용한다는 것을 확인할 수 있다.

In [ ]:
# 상관계수 확인(수치형-원본 데이터)
corrMatt= train[["temp", "atemp", "casual", "registered", "humidity", "windspeed", "count"]]
corrMatt = corrMatt.corr()
print(corrMatt)

mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False

fig, ax = plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
# 상관계수 확인(수치형-randomforest)
corrMatt= train_rand[["temp", "atemp", "casual", "registered", "humidity", "windspeed", "count"]]
corrMatt = corrMatt.corr()
print(corrMatt)

mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False

fig, ax = plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
# 상관계수 확인(수치형-mean)
corrMatt= train_mean[["temp", "atemp", "casual", "registered", "humidity", "windspeed", "count"]]
corrMatt = corrMatt.corr()
print(corrMatt)

mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False

fig, ax = plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

* windspeed의 변화를 주어봤지만 큰 상관이 없는 것으로보임
* register의 경우 높은 상관계수를 보여주지만, test에는 해당 값이 없음

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sns.regplot(x="temp", y="count", data=train,ax=ax1)
sns.regplot(x="windspeed", y="count", data=train,ax=ax2)
sns.regplot(x="humidity", y="count", data=train,ax=ax3)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sns.regplot(x="windspeed", y="count", data=train_rand,ax=ax1)
sns.regplot(x="windspeed", y="count", data=train_mean,ax=ax2)
sns.regplot(x="windspeed", y="count", data=train_rmean,ax=ax3)

* 각 전처리 값에 따라 크게 유의미한 변화는 없는 것으로 보임

In [ ]:
# count 값의 데이터 분포도를 파악

figure, axes = plt.subplots(ncols=2, nrows=2)
figure.set_size_inches(12, 10)

sns.distplot(train["count"], ax=axes[0][0])
stats.probplot(train["count"], dist='norm', fit=True, plot=axes[0][1])
sns.distplot(np.log(train_without_liers["count"]), ax=axes[1][0])
stats.probplot(np.log1p(train_without_liers["count"]), dist='norm', fit=True, plot=axes[1][1])

* count 변수가 오른쪽에 치우쳐져 있다. 대부분의 기계학습은 종속변수가 normal이어야 하기에 정규분포를 갖는 것이 바람직하다.
* 대안으로 outlier data를 제거하고 count 변수에 로그를 씌워 변경해 봐도 정규분포를 따르지는 않지만, 이전 그래프보다는 좀 더 자세히 표현하고 있다.
* ref:https://programmers.co.kr/learn/courses/21/lessons/942

# 4. 분석

## Feature Selection

* 신호와 잡음의 구분 필요
* 연속형 feature = ['temp', 'humidity', 'windspeed', 'atemp']
* 범주형 변수

In [ ]:
# 범주형 변수에 대해 categorical 변수로 변경
categorical_val = ['season', 'holiday', 'workingday', 'weather','dayofweek','month','year','hour']

for var in categorical_val:
    train[var] = train[var].astype('category')
    test[var] = test[var].astype('category')
    train_rand[var] = train_rand[var].astype('category')
    train_mean[var] = train_mean[var].astype('category')

In [ ]:
feature = ['season','weather','temp','atemp','humidity','windspeed',
          'year','hour','dayofweek','holiday','workingday']
feature

In [ ]:
X_train = train[feature]
X_train_rand = train_rand[feature]
X_train_mean = train_mean[feature]

In [ ]:
label_name = 'count'
y_train = train[label_name]
y_train_rand = train_rand[label_name]
y_train_mean = train_mean[label_name]

In [ ]:
X_test = test[feature]
X_test_rand = test[feature]
X_test_mean = test[feature]

# Score
## RMSLE

과대평가 된 항목보다는 과소평가 된 항목에 페널티를 준다.

오차(Error)를 제곱(Square)해서 평균(Mean)한 값의 제곱근(Root)으로 값이 작을수록 정밀도가 높다.

0에 가까운 값이 나올수록 정밀도가 높은 값이다.

Submissions are evaluated one the Root Mean Squared Logarithmic Error (RMSLE)

$$ \sqrt{\frac{1}{n} \sum{i=1}n (\log(pi + 1) - \log(a_i+1))2 } $$

* \({n}\) is the number of hours in the test set
* \(p_i\) is your predicted count
* \(a_i\) is the actual count
* \(\log(x)\) is the natural logarithm

* 좀 더 자세한 설명은 : RMSLE cost function

* 잔차(residual)에 대한 평균에 로그를 씌운 값이다. => 과대평가 된 항목보다 과소평가 된 항목에 페널티를 주기 위해

* 정답에 대한 오류를 숫자로 나타낸 값으로 값이 클수록 오차가 크다는 의미다.

* 값이 작을수록 오류가 적다는 의미를 나타낸다.

In [ ]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)

    # 평균을 낸다.
    mean_difference = difference.mean()

    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)

    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

## Cross validation 교차 검증
* 일반화 정도를 측정하기 위해 여러 데이터를 training / test 로 나눠 반복 학습
* K-fold 교차검증
  * 데이터를 폴드라 부르는 비슷한 크기의 부분집합(n_splits)으로 나누고 각각의 폴드 정확도를 측정한다.
  * 첫 번째 폴드를 테스트 세트로 사용하고 나머지 폴드를 훈련 세트로 사용하여 학습한다.
  * 나머지 훈련 세트로 만들어진 세트의 정확도를 첫 번째 폴드로 평가한다.
  * 다음은 두 번째 폴드가 테스트 세트가 되고 나머지 폴드의 훈련 세트를 두 번째 폴드로 정확도를 측정한다.
  * 이 과정을 마지막 폴드까지 반복한다.
  * 이렇게 훈련 세트와 테스트세트로 나누는 N개의 분할마다 정확도를 측정하여 평균값을 낸 정확도

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 4-1. 분석 - Randomforest
## randomforest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

max_depth_list = []

model = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=0)

model_rand = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=0)

model_mean = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=0)


model

In [ ]:
%time score = cross_val_score(model, X_train, y_train, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

print("Score = {0:.5f}".format(score))

In [ ]:
%time score = cross_val_score(model_rand, X_train_rand, y_train_rand, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

print("Score = {0:.5f}".format(score))

In [ ]:
%time score = cross_val_score(model_mean, X_train_mean, y_train_mean, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

print("Score = {0:.5f}".format(score))

## Train

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model_rand.fit(X_train_rand, y_train_rand)

In [ ]:
model_mean.fit(X_train_mean, y_train_mean)

In [ ]:
# 예측 - train
predictions = model.predict(X_test)
print(predictions.shape)
predictions[0:10]

In [ ]:
# 예측 - rand
predictions_rand = model_rand.predict(X_test_rand)
print(predictions_rand.shape)
predictions_rand[0:10]

In [ ]:
# 예측 - mean
predictions_mean = model_mean.predict(X_test_mean)
print(predictions_mean.shape)
predictions_mean[0:10]

In [ ]:
# 예측한 데이터를 시각화 해본다. 
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.distplot(y_train,ax=ax1,bins=50)
ax1.set(title="train")
sns.distplot(predictions,ax=ax2,bins=50)
ax2.set(title="test")

In [ ]:
# 예측한 데이터를 시각화 해본다. 
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.distplot(y_train_rand,ax=ax1,bins=50)
ax1.set(title="train")
sns.distplot(predictions,ax=ax2,bins=50)
ax2.set(title="test")

In [ ]:
# 예측한 데이터를 시각화 해본다. 
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.distplot(y_train_mean,ax=ax1,bins=50)
ax1.set(title="train")
sns.distplot(predictions,ax=ax2,bins=50)
ax2.set(title="test")

 * 예측 결과 가장 점수가 좋은(낮은 점수)는 randomforest를 통해 돌린 값으로 판단
 * 이에 따라 최종 앙상블을 위한 데이터 세트는 randomforest로 설정하여 수행

In [ ]:
# train dataset 초기화
train = pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
train.shape

In [ ]:
test = pd.read_csv('../input/bike-sharing-demand/test.csv', parse_dates=['datetime'])
test.shape

In [ ]:
train["year"] = train["datetime"].dt.year
train["month"] = train["datetime"].dt.month
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek
train.shape

In [ ]:
test["year"] = test["datetime"].dt.year
test["month"] = test["datetime"].dt.month
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek
test.shape

In [ ]:
train['season'] = train.month.apply(to_right_season)
test['season'] = test.month.apply(to_right_season)

In [ ]:
categorical_feature_names = ["season","holiday","workingday","weather",
                             "dayofweek","month","year","hour"]

In [ ]:
for var in categorical_feature_names:
    train[var] = train[var].astype("category")
    train_rand[var] = train_rand[var].astype('category')
    test[var] = test[var].astype("category")

In [ ]:
feature_names = ["season", "weather", "temp", "atemp", "humidity",
                 "year", "hour", "dayofweek", "holiday", "workingday"
                ]

feature_names

In [ ]:
X_train = train[feature_names]
print(X_train.shape)
X_train.head()

In [ ]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

In [ ]:
label_name = "count"

y_train = train[label_name]

print(y_train.shape)
y_train.head()

In [ ]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values, convertExp=True):

    if convertExp:
        predicted_values = np.exp(predicted_values),
        actual_values = np.exp(actual_values)

    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)

    # 평균을 낸다.
    mean_difference = difference.mean()

    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)

    return score

# 4.2 선형 회귀

* 선형회귀 또는 최소제곱법은 가장 간단하고 오래된 회귀용 선형 알고리즘
* 선형회귀는 예측과 훈련 세트에 있는 타깃 y 사이의 평균 제곱 오차(MSE)를 최소화하는 파라미터 w와 b를 찾는다.
* 매개변수가 없는 것이 장점이지만, 모델의 복잡도를 제어할 수 없다는 단점이 있다.
* 아래 위키피디아에서 가져온 그래프에서 파란 선이 선형회귀 모델을 나타내고 빨간 점들이 훈련데이터를 나타낸다.
* ref : https://youtu.be/woSEc5d_skE

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 선형회귀 모델을 초기화
lModel = LinearRegression()

# 모델을 학습시킨다.
"""
아래의 커널을 참조하여 yLabels를 로그화 하려는데 왜 np.log가 아닌 np.log1p를 활용하는가??
np.log1p는 np.log(1+x)와 동일. 이유는 만약 어떤 x값이 0인데 이를 log하게되면, (-)무한대로 수렴하기 때문에 np.log1p를 활용함. 
참조: https://ko.wikipedia.org/wiki/%EB%A1%9C%EA%B7%B8 
"""
y_train_log = np.log1p(y_train)
lModel.fit(X_train, y_train_log)

# 예측하고 정확도를 평가한다.
preds = lModel.predict(X_train)
print ("RMSLE Value For Linear Regression: ",
       rmsle(np.exp(y_train_log),np.exp(preds), False))

# 4.3. 릿지 - Regularization Model - Ridge

* 회귀를 위한 선형모델
* 가중치(w)의 모든 원소가 0에 가깝게 만들어 모든 피처가 주는 영향을 최소화(기울기를 작게 만듦)
* Regularization(규제)은 오버피팅(과대적합)이 되지 않도록 모델을 강제로 제한한다는 의미
* max_iter(반복 실행하는 최대 횟수)는 3000을 넣어주었다.

* GridSearchCV를 활용하면 우리가 이용하게 될 각각의 모델마다 변경해야 하는 파라미터 튜닝시 어떤 파라미터가 최적의 값을 내는지 등을 알 수 있음.

* GridSearchCV 참조:
  * https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
  * https://datascienceschool.net/view-notebook/ff4b5d491cc34f94aea04baca86fbef8/

In [ ]:
#Ridge모델은 L2제약을 가지는 선형회귀모델에서 개선된 모델이며 해당 모델에서 유의 깊게 튜닝해야하는 파라미터는 alpha값이다.
ridge_m_ = Ridge()
#우리가 튜닝하고자하는 Ridge의 파라미터 중 특정 파라미터에 배열 값으로 넘겨주게 되면 테스트 후 어떤 파라미터가 최적의 값인지 알려줌 
ridge_params_ = { 'max_iter':[3000],'alpha':[0.01, 0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000]}
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_ridge_m = GridSearchCV( ridge_m_,
                          ridge_params_,
                          scoring = rmsle_scorer,
                          cv=5)

y_train_log = np.log1p(y_train)
grid_ridge_m.fit( X_train, y_train_log )
preds = grid_ridge_m.predict(X_train)
print (grid_ridge_m.best_params_)
print ("RMSLE Value For Ridge Regression: ", rmsle(np.exp(y_train_log),np.exp(preds), False))

fig,ax= plt.subplots()
fig.set_size_inches(12,5)
df = pd.DataFrame(grid_ridge_m.cv_results_)
df["alpha"] = df["params"].apply(lambda x:x["alpha"])
df["rmsle"] = df["mean_test_score"].apply(lambda x:-x)

plt.xticks(rotation=30, ha='right')
sns.pointplot(data=df,x="alpha",y="rmsle",ax=ax)

# 4.4. 라쏘 - Regularization Model - Lasso

* 선형 회귀의 Regularization(규제)를 적용하는 대안
* 계수를 0에 가깝게 만들려고 하며, 이를 L1 규제라고 함. 어떤 계수가 0이되기도 하면, 해당 피처는 완전히 제외
* 피처 선택이 자동으로 이루어진다고 볼 수 있음
* alpha 의 default값이 1.0, 과소적합을 줄이기 위해 해당 값을 줄여야 함
* 그리드 서치로 라쏘모델을 실행 했을 때 베스트 알파값은 0.0025
* max_iter(반복 실행하는 최대회수) = 3000으로 설정 
  * ref : https://youtu.be/woSEc5d_skE
  * ref: https://www.kaggle.com/kwonyoung234/for-beginner

In [ ]:
# #Ridge모델은 L1제약을 가지는 선형회귀모델에서 개선된 모델이며 해당 모델에서 유의 깊게 튜닝해야하는 파라미터는 alpha값이다.

lasso_m = Lasso()
alpha = 1/np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
lasso_params = { 'max_iter' : [3000], 'alpha' : alpha}

grid_lasso_m = GridSearchCV( lasso_m, lasso_params, scoring = rmsle_scorer, cv = 5)
y_train_log = np.log1p(y_train)
grid_lasso_m.fit(X_train, y_train_log)
preds = grid_lasso_m.predict(X_train)
print(grid_lasso_m.best_params_)
print('RMSLE Value For Lasso Regression : ', rmsle(np.exp(y_train_log), np.exp(preds), False))

fig, ax = plt.subplots()
fig.set_size_inches(12, 5)
df = pd.DataFrame(grid_lasso_m.cv_results_)
df["alpha"] = df["params"].apply(lambda x:x["alpha"])
df['rmsle'] = df['mean_test_score'].apply(lambda x:-x)

plt.xticks(rotation =30, ha = 'right')
sns.pointplot(data=df, x='alpha', y='rmsle', ax= ax)

# 4.5. - Ensembel Model - RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfModel = RandomForestRegressor(n_estimators=100)

y_train_log = np.log1p(y_train)
rfModel.fit(X_train, y_train_log)

preds = rfModel.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds), False)
print("RMSLE Value for Random Forest : ", score)

# 4.6. Ensemble Model - Gradient Boost

* 여러 개의 결정 트리를 묶어 강력한 모델을 만드는 또 다른 앙상블 기법
* 회귀와 분류에 모두 사용할 수 있음
* 랜덤포레스트와 달리 이진 트리의 오차를 보완하는 방식으로 순차적으로 트리를 만든다.
* 무작위성이 없고 강력한 사전 가지치기가 사용됨
* 1~5개의 깊지 않은 트리를 사용하기 때문에 메모리를 적게 사용하고 예측이 빠름
* learning_rate : 오차를 얼마나 강하게 보정할 것인지를 제어
* n_estimator의 값을 키우면 앙상블에 트리가 더 많이 추가되어 모델의 복잡도가 커지고 훈련 세트에서의 실수를 바로잡을 기회가 많아지지만, 너무 크면 모델이 복잡해지고 오버피팅(과대적합)이 될 수 있다.
* maxdepth(maxleaf_nodes) 복잡도를 너무 높이지 말고 트리의 깊이가 5보다 깊어지지 않게 한다.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators = 4000, alpha=0.01)
y_train_log = np.log1p(y_train)
gbm.fit(X_train, y_train_log)

preds = gbm.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds), False)
print("RMSLE Value For Gradient Boost : " , score)


* 가장 높은 점수는 Randomforest (0.11..)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfModel = RandomForestRegressor(n_estimators=100)

y_train_log = np.log1p(y_train)
rfModel.fit(X_train, y_train_log)

preds = rfModel.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds), False)
print("RMSLE Value for Random Forest : ", score)

# Submission 

In [ ]:
preds_test = rfModel.predict(X_test)
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 6)
sns.distplot(y_train, ax=ax1, bins=50)
sns.distplot(np.exp(preds_test), ax=ax2, bins=50)

In [ ]:
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
submission

submission["count"] = np.exp(preds_test)

print(submission.shape)
submission.head()


In [ ]:
#제출 파일 작성
submission.to_csv("./Score_{0:.5f}_submission.csv".format(score), index=False)

## gradian

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor()
gbm_params={'max_depth':range(1,11,1), 'n_estimators' : [1,10,100]}
grid_gbm=GridSearchCV(gbm, gbm_params, scoring=rmsle_scorer, cv=5)

y_train_log = np.log1p(y_train)
grid_gbm.fit(X_train, y_train_log)
gbm.fit(X_train, y_train_log)

preds = gbm.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds), False)
print("RMSLE Value For Gradient Boost : " , score)



In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators = 4000, alpha=0.01)
y_train_log = np.log1p(y_train)
gbm.fit(X_train, y_train_log)

preds = gbm.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds), False)
print("RMSLE Value For Gradient Boost : " , score)

In [ ]:
# submission 제출파일 작성
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')

submission["count"] = np.exp(preds_test)

print(submission.shape)
submission.head()

In [ ]:
submission.to_csv("./Score_{0:.5f}_submission.csv".format(score), index=False)

* 제출 결과 : 0.413... 
* 크게 높은 점수는 아님